In [2]:
import pyspark
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
spark = pyspark.sql.SparkSession.builder.master("local").getOrCreate()

ratings = spark.read.json('data/ratings.json')
#requests = spark.read.json('data/requests.json')

In [4]:
ratings.show(10)

+--------+------+------------+-------+
|movie_id|rating|   timestamp|user_id|
+--------+------+------------+-------+
|     858|     4|9.56678732E8|   6040|
|    2384|     4|9.56678754E8|   6040|
|     593|     5|9.56678754E8|   6040|
|    1961|     4|9.56678777E8|   6040|
|    1419|     3|9.56678856E8|   6040|
|     213|     5|9.56678856E8|   6040|
|    3111|     5|9.56678856E8|   6040|
|     573|     4|9.56678856E8|   6040|
|    3505|     4|9.56678856E8|   6040|
|    1734|     2|9.56678881E8|   6040|
+--------+------+------------+-------+
only showing top 10 rows



In [5]:
users = spark.read.load("data/users.dat",
     format="csv", sep=":", inferSchema="true").drop('_c1', '_c3', '_c5', '_c7')

users = (users.withColumnRenamed(users.schema.names[0], 'user_id')
              .withColumnRenamed(users.schema.names[1], 'gender')
              .withColumnRenamed(users.schema.names[2], 'age')
              .withColumnRenamed(users.schema.names[3], 'occupation')
              .withColumnRenamed(users.schema.names[4], 'zip'))

In [6]:
users.show(10)

+-------+------+---+----------+-----+
|user_id|gender|age|occupation|  zip|
+-------+------+---+----------+-----+
|      1|     F|  1|        10|48067|
|      2|     M| 56|        16|70072|
|      3|     M| 25|        15|55117|
|      4|     M| 45|         7|02460|
|      5|     M| 25|        20|55455|
|      6|     F| 50|         9|55117|
|      7|     M| 35|         1|06810|
|      8|     M| 25|        12|11413|
|      9|     M| 25|        17|61614|
|     10|     F| 35|         1|95370|
+-------+------+---+----------+-----+
only showing top 10 rows



In [ ]:
# Alternate strategy
# with open('data/users.dat') as f:
#     with open('data/users.csv', 'w') as f2:
#         f2.write('user_id, sex, age, occupation, zip\n')
#         for line in f:
#             f2.write(line.replace('::', ','))
            
# users = spark.read.csv('data/users.csv', header=True)
# users.show(10)

In [32]:
df_raw = users.join(ratings, 'user_id', 'inner').select('gender', 'age', 'occupation', 'zip', 'movie_id', 'rating').toPandas()
df = df_raw.copy()

In [33]:
n = 2
df['zip'] = df_raw['zip'].map(lambda x: str('0'*n)[len(x[:n]):] + x[:n] + 'x' * (5-n))
df['gender'] = df_raw['gender'].map({'M':0, 'F':1})
df['age'] = df_raw['age'].map({1:15, 18:21, 25:30, 35:40, 45:47, 50:53, 56:65})

ohe = OneHotEncoder(sparse=False)
np_ohe = ohe.fit_transform(df[['zip']])

In [34]:
np_df = np.array(df.drop('zip', axis=1))
np_final = np.column_stack((np_df, np_ohe))
ohe_labels = ['zip' + s[s.find('_'):] for s in ohe.get_feature_names()]
col_names = list(df.columns.drop('zip')) + ohe_labels
df_final = pd.DataFrame(np_final, columns=col_names)

In [35]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier

In [36]:
forest = RandomForestClassifier(n_estimators=10, n_jobs=-1)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    df_final.drop('rating', axis=1), 
    df_final['rating'],
    train_size=.1
)

In [40]:
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [43]:
print(forest.score(X_train, y_train), 'is the training score')
forest.score(X_test, y_test)

0.951537628135678 is the training score


0.31170220154177375